In [42]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [44]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=500)

In [45]:
# Carregar o PDF

pdf_link = './weight-loss.pdf'

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

len(pages)

8

In [46]:
# Splitter
child_splitter = RecursiveCharacterTextSplitter(
  chunk_size=200,
)

parent_splitter = RecursiveCharacterTextSplitter(
  chunk_size=4000,
  chunk_overlap=200,
  length_function=len,
  add_start_index=True,
)

In [47]:
# Storages
store = InMemoryStore()
vectorstore = Chroma(
  embedding_function=embedding,
  persist_directory='childVectorDB',
)

In [48]:
parent_document_retriever = ParentDocumentRetriever(
  vectorstore=vectorstore,
  docstore=store,
  child_splitter=child_splitter,
  parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [51]:
TEMPLATE = """
  Você é um especialista em nutrição e saúde. Sua tarefa é responder perguntas sobre o documento fornecido.
  Se a pergunta do usuário não está relacionada ao documento, responda que não sabe.

  Documento:
  {context}

  Pergunta:
  {question}
"""

rag_prompt = ChatPromptTemplate.from_template(
  TEMPLATE,
)


In [52]:
setup_retrieval = RunnableParallel({
  'question': RunnablePassthrough(),
  'context': parent_document_retriever
})

output_parser = StrOutputParser()

In [53]:
parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser

In [54]:
parent_chain_retrieval.invoke('Como perder peso de maneira rápida e saudável?')

'Para perder peso de forma rápida e saudável, é importante seguir algumas diretrizes, como as fornecidas no documento mencionado. Algumas dicas incluem:\n\n1. Consumir menos calorias do que você queima através da dieta e da atividade física.\n2. Aumentar a atividade física enquanto limita a ingestão de calorias para aumentar a taxa de perda de peso.\n3. Manter um diário de alimentos e atividades físicas.\n4. Não pular refeições, especialmente o café da manhã.\n5. Planejar as refeições e comer em horários regulares.\n6. Escolher alimentos ricos em proteínas e fibras, e limitar a ingestão de gordura e açúcar.\n7. Evitar alimentos problemáticos que possam levar a excessos.\n8. Beber bastante líquido, preferencialmente bebidas sem calorias e cafeína.\n9. Dormir adequadamente todas as noites, entre 7-9 horas.\n\nSeguir essas diretrizes, juntamente com a orientação de um nutricionista, pode ajudar na perda de peso de forma saudável e sustentável.'